In [ ]:
#抓取新聞連結
def getlink(script):
    arry = [] 
    global count
    global pkcount 
    apl = 'apl'
    soup = BeautifulSoup(res.text)
    for a_tag in soup.select('#result h2 a'):
        alink = a_tag.get('href').strip()
        arry.append(alink)
        for elelink in arry:
            num = str(pkcount)
            if pkcount < 10:
                num = str(0) * 4 + num
            elif pkcount < 100:     
                num = str(0) * 3 + num
            elif pkcount < 1000:     
                num = str(0) * 2 + num
            elif pkcount < 10000:     
                num = str(0) * 1 + num
            pk = apl + num
        links = {'link':alink,'area':area,'page':page,'count':count,'pk':pk}
        linkary.append(links)
        count += 1
        pkcount += 1

In [ ]:
import requests
from datetime import *
from bs4 import BeautifulSoup
import time

#12個行政區['中正區','大同區','中山區','松山區','大安區','萬華區','信義區','士林區','北投區','內湖區','南港區','文山區']
ary = ['中正區','大同區','中山區','松山區','大安區','萬華區','信義區','士林區','北投區','內湖區','南港區','文山區']
links = {'link':'','area':'','page':'','count':'','pk':''}
linkary =[]
    
#系統時間------------------------------
today = date.today()
year = today.year
month = today.month
day = today.day
if month < 10:
    month = "0" + str(month)
if day < 10:
    day = "0" + str(day)
year = str(year)
month = str(month)
day = str(day)
#--------------------------------------

headers={
'Origin':'http://search.appledaily.com.tw',
'Referer':'http://search.appledaily.com.tw/appledaily/search',
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36' 
}
pkcount = 1    # pkcount = 1
for area in ary:
    count = 1
    for page in range(1,12):
        payload = {
        'searchMode':'',
        'searchType':'text',
        'ExtFilter':'',
        'sorttype':'1',
        'keyword':area,
        'rangedate':'[20030502 TO'+year+month+day+'999:99'']',
        'totalpage':'',
        'page':page
        }

        res = requests.post('http://search.appledaily.com.tw/appledaily/search',headers=headers,data=payload)
        script = res.text
        getlink(script)

In [ ]:
import pandas
data1 = pandas.DataFrame(linkary)
data1

In [ ]:
#輸出成CSV
data1.to_csv('apl_link.csv',encoding='utf-8')

In [ ]:
#讀取CSV檔
import csv
linkary = []
with open('apl_link.csv',) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        linkary.append(row)

In [ ]:
#取得人氣
def getlike(soup): 
    if len(soup.select('.urcc a')) > 0:
        reele = soup.select('.urcc a')[0].text
        print soup.select('.urcc a')
        rdrx = re.search('(\d+)',reele)
        likes = rdrx.group(1)
    elif len(soup.select('.fntss a')) > 0:
        print soup.select('.fntss a')
        reele = soup.select('.fntss a')[0].text
        rdrx = re.search('(\d+)',reele)
        likes = rdrx.group(1)
    else:
        likes = "0"
    return likes 

In [ ]:
#抓取新聞內文
import requests
import time
import re  # --
from bs4 import BeautifulSoup

headers = { 
'Origin':'http://search.appledaily.com.tw',
'Referer':'http://search.appledaily.com.tw/appledaily/search',
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36'
}

for ele in linkary:
    url = ele['link']
    text = ''
    while True:
        try:
            res = requests.get(url,headers=headers)
            res.text
            soup = BeautifulSoup(res.text)
            break
        except:
            time.sleep(5)
            
#     try:
        ele['title'] = soup.select('hgroup #h1')[0].text.strip()
        ele['date'] = soup.select('.gggs time')[0]['datetime']
        for p_tag in soup.select('#summary'):
            text += p_tag.text.strip()
        ele['content'] = text
        ele['likes'] = getlike(soup)
        
#     except:
#         print "This link has some problems!"
#         print ele['link'], ele['area'], ele['count']

In [ ]:
#新聞內文進入DataFrame
import pandas
data2 = pandas.DataFrame(linkary)

# 移除多餘的 column
# del data2['column name']
apl_ok = data2.drop(data2.columns[[0]], axis=1)
apl_ok

In [ ]:
#輸出完成品
apl_ok.to_csv('apl.csv',encoding='utf-8')